In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_tam
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/transliterated/'

    train_data = read_task(dataset_location , split = 'tam_train_trans')
    dev_data = read_task(dataset_location , split = 'tam_dev_trans')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_tam
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['bert-base-multilingual-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Tamil' + 'transliterated' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHE

FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.2232643365859985
Validation loss per 100 evaluation steps: 0.9995421035455005
DEV A

FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.127055287361145
Validation loss per 100 evaluation steps: 1.4069451854960753
DEV ACC: 0.6244184243176178
DEV Precision: 0.47712016823708403
DEV Recall: 0.44230930632391235
DEV F1Score: 0.43206797748814296
BEST ACCURACY -->  DEV: 0.65105
BEST PRECISION -->  DEV: 0.51146
BEST RECALL -->  DEV: 0.47161
BEST F1SCORE -->  DEV: 0.4615
TIME PER EPOCH: 29.853833301862082

Training epoch: 10
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED B

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

...

FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.0189799070358276
Validation loss per 100 evaluation steps: 1.4204010438210894
DEV ACC: 0.5924123759305211
DEV Precision: 0.4436612856629085
DEV Recall: 0.46606643764369005
DEV F1Score: 0.4338476834950519
BEST ACCURACY -->  DEV: 0.64716
BEST PRECISION -->  DEV: 0.49256
BEST RECALL -->  DEV: 0.47968
BEST F1SCORE -->  DEV: 0.46162
TIME PER EPOCH: 29.48638877471288

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHE

FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1

Validation loss per 100 evaluation steps: 1.4273664473307015
DEV ACC: 0.6270161290322581
DEV Precision: 0.49950456493433043
DEV Recall: 0.472686247781961
DEV F1Score: 0.4604553761092361
BEST ACCURACY -->  DEV: 0.64528
BEST PRECISION -->  DEV: 0.4995
BEST RECALL -->  DEV: 0.49206
BEST F1SCORE -->  DEV: 0.46373
TIME PER EPOCH: 29.98099667628606

Training epoch: 10
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1

In [6]:
print(best_dev_acc)

0.6452776054590571


In [7]:
print(best_epoch)

4


In [8]:
print(best_dev_precision)

0.49950456493433043


In [9]:
print(best_dev_recall)

0.49205808394816714


In [10]:
print(best_dev_f1score)

0.46372697741312136


In [11]:
print(overall_list_dev_acc)

[[0.6307769851116626, 0.6422340260545906, 0.6510545905707196, 0.6507444168734492, 0.6358366935483871, 0.6298270781637717, 0.6234685173697271, 0.6188352977667494, 0.6244184243176178, 0.590919665012407], [0.6049550248138958, 0.6443858560794046, 0.6459754962779156, 0.6462275124069479, 0.6189903846153846, 0.6082893920595533, 0.6065640508684863, 0.6307188275434243, 0.6153846153846154, 0.6093556141439207], [0.6278884925558313, 0.6414973635235732, 0.6343633684863523, 0.6346153846153846, 0.636883529776675, 0.6388415012406948, 0.6283925248138958, 0.6013492555831266, 0.6028613523573201, 0.5938081575682381], [0.6225961538461539, 0.6418075372208436, 0.6471580334987593, 0.6414973635235732, 0.6251744727047146, 0.6009809243176178, 0.6176721464019852, 0.6241082506203475, 0.5924123759305211, 0.6020665322580645], [0.6225379962779156, 0.6452776054590571, 0.6439593672456575, 0.6310290012406948, 0.6314748759305211, 0.6358173076923077, 0.6223441377171216, 0.6197270471464019, 0.6270161290322581, 0.5896983560

In [12]:
print(overall_list_dev_precision)

[[0.4486315485683587, 0.4970028146386442, 0.5114558314789559, 0.5097981956593098, 0.48395324106337584, 0.5016961712619394, 0.4671890789657174, 0.4769676461734034, 0.47712016823708403, 0.47413286950953804], [0.4371630105711197, 0.4724841173983531, 0.4964778327503392, 0.4997453153212899, 0.4893192946187824, 0.4717977475283955, 0.49036530815111956, 0.4701932343186689, 0.47438405559589486, 0.4734590157395424], [0.47515265210889307, 0.5093833703771163, 0.49271304019418793, 0.4895500731212851, 0.501270112068934, 0.4883355149338809, 0.4852658809040606, 0.4759274614794401, 0.4840048137456025, 0.4810829444675415], [0.4481818270614683, 0.4739143847672543, 0.4816413362670591, 0.49256450456144973, 0.48334004310281414, 0.48461475412846167, 0.47971980433020683, 0.4760446472642111, 0.4436612856629085, 0.4852163251748757], [0.44562287218048624, 0.4934303264932591, 0.49600575112131523, 0.4707195087548788, 0.49350164187092826, 0.49815045950819187, 0.4695801297736349, 0.48101516547673645, 0.4995045649343

In [13]:
print(overall_list_dev_recall)

[[0.37468712852486336, 0.4307099611055175, 0.46284528328921004, 0.471609862500526, 0.4600991278492274, 0.46764702336316577, 0.44867345444263645, 0.4492852307618184, 0.44230930632391235, 0.4648116165066383], [0.44137622771410895, 0.44467274097183557, 0.4467454547527461, 0.4640566015455805, 0.49179449765911615, 0.4734201138818992, 0.4778183155915942, 0.4424118632181404, 0.46746251775178377, 0.4560628074963291], [0.4236563823839768, 0.4758683184359151, 0.49222140576553597, 0.47188741813335133, 0.44819336809475707, 0.4639379208330769, 0.4576096639876022, 0.47233463462305053, 0.48139647005467073, 0.4891389529053963], [0.4239238744251345, 0.43170456967623455, 0.45800116067938146, 0.4627492424979949, 0.4682981747939969, 0.4796817140975097, 0.46686405387178614, 0.4546565993579035, 0.46606643764369005, 0.4641159245994705], [0.4204840333866153, 0.46287858446134506, 0.4584143361247797, 0.4405695796415281, 0.49205808394816714, 0.4649610302292161, 0.4553572089009293, 0.4521042839724356, 0.472686247

In [14]:
print(overall_list_dev_f1score)

[[0.3717948708349509, 0.42212432184417553, 0.4532795969045006, 0.4614954847864319, 0.44745165439459433, 0.46088662815367265, 0.4351463791210556, 0.43645764716484403, 0.43206797748814296, 0.4424553202705544], [0.3990337681757862, 0.41943238598277055, 0.43544652320589883, 0.44845075146305, 0.46589122365277463, 0.44870728655475167, 0.4612125100492519, 0.4316589903543657, 0.44600190360526193, 0.4408841079583263], [0.41063753388340174, 0.45792703017858044, 0.45736916923572185, 0.45494323613398063, 0.4434558158821381, 0.4510556176413138, 0.44641975169551174, 0.4468917208420226, 0.4575591037924851, 0.4597555302927952], [0.4038390499768625, 0.418656465075116, 0.43822708820080775, 0.4484553274621516, 0.45200492121142366, 0.46161707278854053, 0.4452934774913181, 0.44408564765131514, 0.4338476834950519, 0.4497503335788233], [0.4029670157837685, 0.4488786154696157, 0.4447773564710044, 0.42365568586996416, 0.46372697741312136, 0.4554981665804951, 0.4355164817288038, 0.4401676697045889, 0.4604553761

In [15]:
#The best model is 0